### check class distribution to decide overlap for windows

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# =====================================================
# LOAD AND ANALYZE DATA
# =====================================================

RAW_DATA_FILE = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/combined_all.csv'
FS = 125

print("="*70)
print("ANALYZING 10-CLASS ACTIVITY DISTRIBUTION")
print("="*70)

# Load data
df = pd.read_csv(RAW_DATA_FILE)
print(f"\n✓ Loaded {len(df)} rows")

# =====================================================
# 1. RECORDINGS PER ACTIVITY
# =====================================================

print(f"\n" + "="*70)
print("1. RECORDINGS PER ACTIVITY")
print("="*70)

# Group by activity and subject to count recordings
recordings_per_activity = df.groupby(['activity_id', 'activity_label', 'subject']).size().reset_index(name='samples')

# Count unique subjects per activity
subjects_per_activity = recordings_per_activity.groupby(['activity_id', 'activity_label'])['subject'].nunique()

print(f"\nNumber of subjects (recordings) per activity:")
for (activity_id, activity_label), n_subjects in subjects_per_activity.items():
    print(f"  Activity {activity_id:2d} ({activity_label:20s}): {n_subjects} subjects")

# Which activities have 6 recordings vs 5?
activities_6_recordings = subjects_per_activity[subjects_per_activity == 6].index.get_level_values('activity_id').tolist()
activities_5_recordings = subjects_per_activity[subjects_per_activity == 5].index.get_level_values('activity_id').tolist()

print(f"\nSummary:")
print(f"  Activities with 6 recordings (all subjects): {activities_6_recordings}")
print(f"  Activities with 5 recordings (missing Subject 5): {activities_5_recordings}")

# =====================================================
# 2. TOTAL DURATION PER ACTIVITY
# =====================================================

print(f"\n" + "="*70)
print("2. TOTAL DURATION PER ACTIVITY")
print("="*70)

duration_per_activity = df.groupby(['activity_id', 'activity_label']).size().apply(lambda x: x / FS)

print(f"\nTotal duration per activity:")
for (activity_id, activity_label), duration in duration_per_activity.items():
    is_6_rec = "✓" if activity_id in activities_6_recordings else " "
    print(f"  {is_6_rec} Activity {activity_id:2d} ({activity_label:20s}): {duration:6.1f}s ({duration/60:5.2f} min)")

print(f"\nStatistics:")
print(f"  Mean:   {duration_per_activity.mean():.1f}s")
print(f"  Std:    {duration_per_activity.std():.1f}s")
print(f"  Min:    {duration_per_activity.min():.1f}s")
print(f"  Max:    {duration_per_activity.max():.1f}s")
print(f"  Ratio (max/min): {duration_per_activity.max() / duration_per_activity.min():.2f}x")

# =====================================================
# 3. DURATION PER RECORDING (VARIABILITY)
# =====================================================

print(f"\n" + "="*70)
print("3. DURATION PER RECORDING (VARIABILITY)")
print("="*70)

recording_durations = df.groupby(['activity_id', 'activity_label', 'subject']).size().apply(lambda x: x / FS)

print(f"\nMean recording duration per activity:")
for activity_id in sorted(df['activity_id'].unique()):
    activity_label = df[df['activity_id'] == activity_id]['activity_label'].iloc[0]
    activity_recs = recording_durations[activity_id]
    
    mean_dur = activity_recs.mean()
    std_dur = activity_recs.std()
    min_dur = activity_recs.min()
    max_dur = activity_recs.max()
    
    print(f"  Activity {activity_id:2d} ({activity_label:20s}): "
          f"{mean_dur:5.1f}s ± {std_dur:4.1f}s  "
          f"(range: {min_dur:.1f}s - {max_dur:.1f}s)")

# =====================================================
# 4. ESTIMATED WINDOWS WITH DIFFERENT OVERLAPS
# =====================================================

print(f"\n" + "="*70)
print("4. ESTIMATED WINDOWS WITH DIFFERENT OVERLAPS")
print("="*70)

WINDOW_SIZE = 4  # seconds
WINDOW_SAMPLES = int(WINDOW_SIZE * FS)

def estimate_windows(duration_sec, overlap_pct):
    """Estimate number of windows from duration."""
    stride = WINDOW_SIZE * (1 - overlap_pct)
    n_windows = int((duration_sec - WINDOW_SIZE) / stride) + 1
    return max(0, n_windows)

overlaps_to_test = [0.0, 0.25, 0.5, 0.75]

for overlap in overlaps_to_test:
    print(f"\nWith {overlap*100:.0f}% overlap:")
    
    windows_per_activity = {}
    for (activity_id, activity_label), duration in duration_per_activity.items():
        n_windows = estimate_windows(duration, overlap)
        windows_per_activity[activity_id] = n_windows
        print(f"  Activity {activity_id:2d} ({activity_label:20s}): ~{n_windows:4d} windows")
    
    total_windows = sum(windows_per_activity.values())
    min_windows = min(windows_per_activity.values())
    max_windows = max(windows_per_activity.values())
    
    print(f"  Total: {total_windows} windows")
    print(f"  Imbalance ratio (max/min): {max_windows}/{min_windows} = {max_windows/min_windows:.2f}x")

# =====================================================
# 5. VISUALIZATION
# =====================================================

print(f"\n" + "="*70)
print("5. GENERATING VISUALIZATION")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Number of subjects per activity
ax1 = axes[0, 0]
activity_ids = sorted(df['activity_id'].unique())
n_subjects = [subjects_per_activity.loc[aid].iloc[0] if len(subjects_per_activity.loc[aid].shape) > 0 
              else subjects_per_activity.loc[aid] for aid in activity_ids]
colors = ['green' if n == 6 else 'orange' for n in n_subjects]
bars = ax1.bar(activity_ids, n_subjects, color=colors)
ax1.set_xlabel('Activity ID', fontsize=12)
ax1.set_ylabel('Number of Subjects (Recordings)', fontsize=12)
ax1.set_title('Recordings per Activity', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 7)
ax1.set_xticks(activity_ids)
ax1.grid(True, alpha=0.3, axis='y')
ax1.axhline(y=5, color='orange', linestyle='--', alpha=0.5, label='5 subjects')
ax1.axhline(y=6, color='green', linestyle='--', alpha=0.5, label='6 subjects')
ax1.legend()

# Plot 2: Total duration per activity
ax2 = axes[0, 1]
durations = [duration_per_activity.loc[aid].iloc[0] if len(duration_per_activity.loc[aid].shape) > 0 
             else duration_per_activity.loc[aid] for aid in activity_ids]
colors = ['green' if aid in activities_6_recordings else 'orange' for aid in activity_ids]
ax2.bar(activity_ids, durations, color=colors)
ax2.set_xlabel('Activity ID', fontsize=12)
ax2.set_ylabel('Total Duration (seconds)', fontsize=12)
ax2.set_title('Total Duration per Activity', fontsize=14, fontweight='bold')
ax2.set_xticks(activity_ids)
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Windows per activity with different overlaps
ax3 = axes[1, 0]
width = 0.2
x = np.arange(len(activity_ids))
for i, overlap in enumerate(overlaps_to_test):
    windows = []
    for aid in activity_ids:
        dur = duration_per_activity.loc[aid].iloc[0] if len(duration_per_activity.loc[aid].shape) > 0 else duration_per_activity.loc[aid]
        windows.append(estimate_windows(dur, overlap))
    offset = (i - len(overlaps_to_test)/2 + 0.5) * width
    ax3.bar(x + offset, windows, width, label=f'{overlap*100:.0f}% overlap', alpha=0.8)

ax3.set_xlabel('Activity ID', fontsize=12)
ax3.set_ylabel('Number of Windows', fontsize=12)
ax3.set_title('Estimated Windows per Activity (Different Overlaps)', fontsize=14, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(activity_ids)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Class imbalance ratio vs overlap
ax4 = axes[1, 1]
overlap_range = np.linspace(0, 0.9, 50)
imbalance_ratios = []
for overlap in overlap_range:
    windows = []
    for aid in activity_ids:
        dur = duration_per_activity.loc[aid].iloc[0] if len(duration_per_activity.loc[aid].shape) > 0 else duration_per_activity.loc[aid]
        windows.append(estimate_windows(dur, overlap))
    ratio = max(windows) / min(windows)
    imbalance_ratios.append(ratio)

ax4.plot(overlap_range * 100, imbalance_ratios, linewidth=2, color='steelblue')
ax4.axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% overlap (your plan)')
ax4.set_xlabel('Overlap (%)', fontsize=12)
ax4.set_ylabel('Imbalance Ratio (max/min)', fontsize=12)
ax4.set_title('Class Imbalance vs Overlap', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.legend()

plt.tight_layout()
plt.savefig('activity_distribution_analysis.png', dpi=150, bbox_inches='tight')
print(f"\n✓ Saved visualization: activity_distribution_analysis.png")

plt.show()

# =====================================================
# 6. RECOMMENDATIONS
# =====================================================

print(f"\n" + "="*70)
print("6. RECOMMENDATIONS")
print("="*70)

# Calculate imbalance with 50% overlap
windows_50pct = {}
for aid in activity_ids:
    dur = duration_per_activity.loc[aid].iloc[0] if len(duration_per_activity.loc[aid].shape) > 0 else duration_per_activity.loc[aid]
    windows_50pct[aid] = estimate_windows(dur, 0.5)

min_windows_50 = min(windows_50pct.values())
max_windows_50 = max(windows_50pct.values())
imbalance_50 = max_windows_50 / min_windows_50

print(f"\nWith 50% overlap:")
print(f"  Min windows: {min_windows_50}")
print(f"  Max windows: {max_windows_50}")
print(f"  Imbalance ratio: {imbalance_50:.2f}x")

if imbalance_50 < 1.5:
    print(f"\n✓ Imbalance is ACCEPTABLE (<1.5x)")
    print(f"  Recommendation: Use 50% overlap + class weights")
elif imbalance_50 < 2.0:
    print(f"\n⚠️  Imbalance is MODERATE (1.5-2.0x)")
    print(f"  Recommendation: Use 50% overlap + class weights (should work)")
    print(f"  Alternative: Consider differential overlap to balance")
else:
    print(f"\n✗ Imbalance is SEVERE (>2.0x)")
    print(f"  Recommendation: Use differential overlap to balance classes")
    print(f"  Alternative: Use 50% + class weights + monitor per-class performance")

print(f"\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)

## Preprocessing raw data for multi class classification
1. z score per channel
2. change point detection to get segments
3. segment based train test split with buffer
4. 4s windows with 50% overlap for all class

In [3]:
import pandas as pd
import numpy as np
import pickle
import os
import ruptures as rpt
from tqdm import tqdm
import gc

# =====================================================
# CONFIGURATION
# =====================================================

RAW_DATA_FILE = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/combined_all.csv'
OUTPUT_DIR = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/windowed_zscore_temporal5fold_50overlap_10class'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# CPD parameters (same as before)
FS = 125
CPD_MODEL = 'l2'
CPD_MIN_SIZE = int(4 * FS)
CPD_JUMP = 5
CPD_PEN = 10

# Windowing parameters
WINDOW_SIZE = 4
WINDOW_SAMPLES = int(WINDOW_SIZE * FS)

# Cross-validation
N_FOLDS = 5
ACCEPTABLE_RANGE = (0.18, 0.22)

# Overlap configuration - 50% for ALL classes
OVERLAP = 0.5

print("="*70)
print("PIPELINE: RAW → Z-SCORE → CPD → 5-FOLD → WINDOWING (10-CLASS)")
print("="*70)

print(f"\nConfiguration:")
print(f"  Classes: 10 (multi-class)")
print(f"  CPD: model={CPD_MODEL}, min_size={CPD_MIN_SIZE} samples (4s)")
print(f"  Window: {WINDOW_SIZE}s ({WINDOW_SAMPLES} samples)")
print(f"  CV: {N_FOLDS}-fold temporal split")
print(f"  Test range: {ACCEPTABLE_RANGE[0]*100:.0f}%-{ACCEPTABLE_RANGE[1]*100:.0f}%")
print(f"  Overlap: {OVERLAP*100:.0f}% for ALL classes")

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def detect_change_points(data, min_size, pen, jump):
    """Detect change points using Pelt algorithm with L2 cost."""
    signal_variance = np.var(data, axis=1)
    algo = rpt.Pelt(model='l2', min_size=min_size, jump=jump)
    algo.fit(signal_variance.reshape(-1, 1))
    breakpoints = algo.predict(pen=pen)
    return breakpoints


def create_windows_from_segment(segment, window_size, overlap):
    """Create windows from ONE segment with specified overlap."""
    n_samples, n_channels = segment.shape
    stride = int(window_size * (1 - overlap))
    
    windows = []
    start = 0
    
    while start + window_size <= n_samples:
        window = segment[start:start + window_size, :]
        windows.append(window)
        start += stride
    
    return np.array(windows) if len(windows) > 0 else np.array([]).reshape(0, window_size, n_channels)


def select_test_segments_full_enforcement(segments, fold, n_folds, 
                                         acceptable_range=(0.18, 0.22)):
    """FULL ENFORCEMENT: Guarantees test percentage within acceptable_range."""
    total_duration = sum(seg['duration_sec'] for seg in segments)
    
    start_pct = (fold - 1) / n_folds
    end_pct = fold / n_folds
    target_start_time = total_duration * start_pct
    target_end_time = total_duration * end_pct
    
    test_indices = []
    test_duration = 0
    buffer_indices = []
    
    cumulative_time = 0
    for idx, seg in enumerate(segments):
        seg_start = cumulative_time
        seg_end = cumulative_time + seg['duration_sec']
        
        if seg_end > target_start_time and seg_start < target_end_time:
            new_test_duration = test_duration + seg['duration_sec']
            new_test_pct = new_test_duration / total_duration
            
            if new_test_pct <= acceptable_range[1]:
                test_indices.append(idx)
                test_duration = new_test_duration
            else:
                buffer_indices.append(idx)
                break
        
        cumulative_time = seg_end
    
    final_test_pct = test_duration / total_duration if total_duration > 0 else 0
    
    if final_test_pct < acceptable_range[0]:
        if len(buffer_indices) > 0:
            buffer_seg = buffer_indices[0]
            new_test_duration = test_duration + segments[buffer_seg]['duration_sec']
            new_test_pct = new_test_duration / total_duration
            
            dist_without = acceptable_range[0] - final_test_pct
            dist_with = new_test_pct - acceptable_range[1] if new_test_pct > acceptable_range[1] else 0
            
            if dist_with <= dist_without:
                test_indices.append(buffer_seg)
                test_duration = new_test_duration
                buffer_indices = []
                final_test_pct = new_test_pct
    
    train_indices = [i for i in range(len(segments)) 
                    if i not in test_indices and i not in buffer_indices]
    
    if fold == n_folds and len(train_indices) > 0 and len(test_indices) > 0:
        max_train = max(train_indices)
        min_test = min(test_indices)
        
        if max_train + 1 == min_test and len(buffer_indices) == 0:
            test_pct_current = test_duration / total_duration
            
            if test_pct_current >= acceptable_range[0]:
                train_indices.remove(max_train)
                buffer_indices.append(max_train)
    
    test_duration = sum(segments[i]['duration_sec'] for i in test_indices)
    train_duration = sum(segments[i]['duration_sec'] for i in train_indices)
    buffer_duration = sum(segments[i]['duration_sec'] for i in buffer_indices)
    
    actual_test_pct = test_duration / total_duration if total_duration > 0 else 0
    actual_train_pct = train_duration / total_duration if total_duration > 0 else 0
    actual_buffer_pct = buffer_duration / total_duration if total_duration > 0 else 0
    
    buffer_applied = len(buffer_indices) > 0
    
    return test_indices, train_indices, buffer_indices, actual_test_pct, actual_train_pct, actual_buffer_pct, buffer_applied


def create_windows_for_fold(fold, segments_by_recording, fold_info_df, overlap):
    """Create windows with uniform overlap for all classes."""
    train_windows = []
    train_labels = []
    val_windows = []
    val_labels = []
    
    fold_recordings = fold_info_df[fold_info_df['fold'] == fold]
    
    for _, rec_fold_info in fold_recordings.iterrows():
        rec_id = rec_fold_info['recording_id']
        recording_segments = segments_by_recording[rec_id]
        
        # Train segments
        for seg_idx in rec_fold_info['train_segment_indices']:
            segment = recording_segments[seg_idx]
            label = segment['activity_id']
            
            windows = create_windows_from_segment(segment['segment_data'], WINDOW_SAMPLES, overlap)
            
            if len(windows) > 0:
                train_windows.extend(windows)
                train_labels.extend([label] * len(windows))
        
        # Test segments
        for seg_idx in rec_fold_info['test_segment_indices']:
            segment = recording_segments[seg_idx]
            label = segment['activity_id']
            
            windows = create_windows_from_segment(segment['segment_data'], WINDOW_SAMPLES, overlap)
            
            if len(windows) > 0:
                val_windows.extend(windows)
                val_labels.extend([label] * len(windows))
    
    return np.array(train_windows), np.array(train_labels), np.array(val_windows), np.array(val_labels)

# =====================================================
# STEP 1: LOAD RAW DATA
# =====================================================

print(f"\n" + "="*70)
print("STEP 1: LOAD RAW DATA")
print("="*70)

df = pd.read_csv(RAW_DATA_FILE)
print(f"✓ Loaded {len(df)} rows")

eeg_columns = [f'ch{i}' for i in range(1, 17)]
grouped = df.groupby(['subject', 'activity_id', 'activity_label'])

raw_recordings = []

for (subject, activity_id, activity_label), group in tqdm(grouped, desc="Creating recordings"):
    eeg_data = group[eeg_columns].values
    duration_sec = len(eeg_data) / FS
    
    raw_recordings.append({
        'subject': subject,
        'activity_id': activity_id,
        'activity_label': activity_label,
        'data': eeg_data,
        'duration_sec': duration_sec
    })

print(f"\n✓ Created {len(raw_recordings)} raw recordings")

subjects_in_data = sorted(list(set([item['subject'] for item in raw_recordings])))
activities_in_data = sorted(list(set([item['activity_id'] for item in raw_recordings])))
print(f"  Subjects: {subjects_in_data}")
print(f"  Activities: {activities_in_data}")

total_dataset_duration = sum(item['duration_sec'] for item in raw_recordings)
print(f"\nDataset: {total_dataset_duration:.1f}s ({total_dataset_duration/60:.1f} min)")

# =====================================================
# STEP 2: Z-SCORE NORMALIZATION
# =====================================================

print(f"\n" + "="*70)
print("STEP 2: Z-SCORE NORMALIZATION")
print("="*70)

for recording in tqdm(raw_recordings, desc="Normalizing"):
    data = recording['data']
    
    mean_per_channel = np.mean(data, axis=0, keepdims=True)
    std_per_channel = np.std(data, axis=0, keepdims=True) + 1e-8
    
    normalized_data = (data - mean_per_channel) / std_per_channel
    recording['data'] = normalized_data

print(f"✓ Normalized {len(raw_recordings)} recordings (z-score per channel)")

# =====================================================
# STEP 3: CHANGE POINT DETECTION
# =====================================================

print(f"\n" + "="*70)
print("STEP 3: CHANGE POINT DETECTION")
print("="*70)

all_segments = []
segment_counter = 0

for rec_idx, recording in enumerate(tqdm(raw_recordings, desc="Running CPD")):
    subject = recording['subject']
    activity_id = recording['activity_id']
    activity_label = recording['activity_label']
    data = recording['data']
    
    try:
        breakpoints = detect_change_points(data, CPD_MIN_SIZE, CPD_PEN, CPD_JUMP)
        
        start = 0
        for end in breakpoints:
            segment_data = data[start:end, :]
            segment_duration = (end - start) / FS
            
            all_segments.append({
                'segment_id': segment_counter,
                'recording_id': rec_idx,
                'subject': subject,
                'activity_id': activity_id,
                'activity_label': activity_label,
                'segment_data': segment_data,
                'n_samples': end - start,
                'duration_sec': segment_duration,
                'start_time_in_recording': start / FS
            })
            segment_counter += 1
            start = end
        
    except Exception as e:
        print(f"\n✗ CPD failed for {subject}, Activity {activity_id}: {e}")
        continue

print(f"\n✓ Created {len(all_segments)} segments")

segments_df = pd.DataFrame([
    {k: v for k, v in seg.items() if k != 'segment_data'}
    for seg in all_segments
])

print(f"\nSegment summary:")
print(f"  Mean: {segments_df['duration_sec'].mean():.1f}s ± {segments_df['duration_sec'].std():.1f}s")
print(f"  Range: {segments_df['duration_sec'].min():.1f}s - {segments_df['duration_sec'].max():.1f}s")

segments_metadata_file = os.path.join(OUTPUT_DIR, 'segments_metadata.csv')
segments_df.to_csv(segments_metadata_file, index=False)

# =====================================================
# STEP 4: TEMPORAL 5-FOLD SPLIT
# =====================================================

print(f"\n" + "="*70)
print("STEP 4: TEMPORAL 5-FOLD SPLIT")
print("="*70)

segments_by_recording = {}
for seg in all_segments:
    rec_id = seg['recording_id']
    if rec_id not in segments_by_recording:
        segments_by_recording[rec_id] = []
    segments_by_recording[rec_id].append(seg)

fold_info_per_recording = []
total_buffer_segments = 0

for rec_id in range(len(raw_recordings)):
    recording_segments = segments_by_recording.get(rec_id, [])
    
    if len(recording_segments) == 0:
        continue
    
    recording = raw_recordings[rec_id]
    
    for fold in range(1, N_FOLDS + 1):
        test_indices, train_indices, buffer_indices, test_pct, train_pct, buffer_pct, buffer_applied = \
            select_test_segments_full_enforcement(
                recording_segments,
                fold,
                N_FOLDS,
                acceptable_range=ACCEPTABLE_RANGE
            )
        
        total_buffer_segments += len(buffer_indices)
        
        fold_info_per_recording.append({
            'recording_id': rec_id,
            'subject': recording['subject'],
            'activity_id': recording['activity_id'],
            'activity_label': recording['activity_label'],
            'fold': fold,
            'test_segment_indices': test_indices,
            'train_segment_indices': train_indices,
            'buffer_segment_indices': buffer_indices,
            'buffer_applied': buffer_applied,
            'n_test_segments': len(test_indices),
            'n_train_segments': len(train_indices),
            'n_buffer_segments': len(buffer_indices),
            'test_duration': sum(recording_segments[i]['duration_sec'] for i in test_indices),
            'train_duration': sum(recording_segments[i]['duration_sec'] for i in train_indices),
            'buffer_duration': sum(recording_segments[i]['duration_sec'] for i in buffer_indices),
            'recording_duration': recording['duration_sec'],
            'test_pct': test_pct * 100,
            'train_pct': train_pct * 100,
            'buffer_pct': buffer_pct * 100
        })

fold_info_df = pd.DataFrame(fold_info_per_recording)

print(f"\n✓ Created {N_FOLDS}-fold split")
print(f"✓ Total buffer segments: {total_buffer_segments}")

print(f"\nGLOBAL percentages:")
range_violations = 0
for fold in range(1, N_FOLDS + 1):
    fold_data = fold_info_df[fold_info_df['fold'] == fold]
    total_test = fold_data['test_duration'].sum()
    global_test_pct = (total_test / total_dataset_duration) * 100
    
    in_range = ACCEPTABLE_RANGE[0]*100 <= global_test_pct <= ACCEPTABLE_RANGE[1]*100
    status = "✓" if in_range else "✗"
    
    if not in_range:
        range_violations += 1
    
    print(f"  Fold {fold}: Test={global_test_pct:.1f}% {status}")

if range_violations == 0:
    print(f"\n✓ ALL FOLDS WITHIN RANGE!")

fold_info_file = os.path.join(OUTPUT_DIR, 'fold_info_per_recording.csv')
fold_info_df.to_csv(fold_info_file, index=False)

# =====================================================
# STEP 5: CREATE WINDOWS
# =====================================================

print(f"\n" + "="*70)
print("STEP 5: CREATE WINDOWS")
print("="*70)

cv_splits = []

for fold in range(1, N_FOLDS + 1):
    print(f"\nFold {fold}/{N_FOLDS}...", end=" ")
    
    X_train, y_train, X_val, y_val = create_windows_for_fold(
        fold, 
        segments_by_recording, 
        fold_info_df,
        overlap=OVERLAP
    )
    
    # Activity distribution in this fold
    train_activity_counts = {aid: np.sum(y_train == aid) for aid in np.unique(y_train)}
    val_activity_counts = {aid: np.sum(y_val == aid) for aid in np.unique(y_val)}
    
    print(f"Train: {len(X_train)}, Val: {len(X_val)}")
    
    # Show class distribution
    print(f"    Train distribution:")
    for aid in sorted(train_activity_counts.keys()):
        count = train_activity_counts[aid]
        pct = count / len(y_train) * 100
        print(f"      Activity {aid:2d}: {count:4d} ({pct:4.1f}%)")
    
    cv_splits.append({
        'fold': fold,
        'X_train': X_train,
        'y_train': y_train,
        'X_val': X_val,
        'y_val': y_val,
        'n_test_windows': len(X_val),
        'n_train_windows': len(X_train),
        'train_activity_counts': train_activity_counts,
        'val_activity_counts': val_activity_counts
    })
    
    del X_train, X_val
    gc.collect()

cv_file = os.path.join(OUTPUT_DIR, 'cv_splits.pkl')
with open(cv_file, 'wb') as f:
    pickle.dump(cv_splits, f)
print(f"\n✓ Saved: {cv_file}")

# =====================================================
# SAVE METADATA
# =====================================================

metadata = {
    'label_type': 'multi_class',
    'n_classes': len(activities_in_data),
    'activity_ids': activities_in_data,
    'window_size_sec': WINDOW_SIZE,
    'window_samples': WINDOW_SAMPLES,
    'overlap': OVERLAP,
    'sampling_rate': FS,
    'n_channels': 16,
    'n_folds': N_FOLDS,
    'preprocessing': 'z_score_normalization_per_channel',
    'cpd_params': {
        'model': CPD_MODEL,
        'min_size': CPD_MIN_SIZE,
        'jump': CPD_JUMP,
        'penalty': CPD_PEN
    }
}

metadata_file = os.path.join(OUTPUT_DIR, 'dataset_metadata.pkl')
with open(metadata_file, 'wb') as f:
    pickle.dump(metadata, f)

print(f"\n" + "="*70)
print("COMPLETED")
print("="*70)
print(f"\nOutput: {OUTPUT_DIR}")

PIPELINE: RAW → Z-SCORE → CPD → 5-FOLD → WINDOWING (10-CLASS)

Configuration:
  Classes: 10 (multi-class)
  CPD: model=l2, min_size=500 samples (4s)
  Window: 4s (500 samples)
  CV: 5-fold temporal split
  Test range: 18%-22%
  Overlap: 50% for ALL classes

STEP 1: LOAD RAW DATA
✓ Loaded 3079330 rows


Creating recordings: 100%|██████████| 54/54 [00:00<00:00, 123.81it/s]



✓ Created 54 raw recordings
  Subjects: ['s1', 's2', 's3', 's4', 's5', 's6']
  Activities: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]

Dataset: 24634.6s (410.6 min)

STEP 2: Z-SCORE NORMALIZATION


Normalizing: 100%|██████████| 54/54 [00:00<00:00, 573.17it/s]


✓ Normalized 54 recordings (z-score per channel)

STEP 3: CHANGE POINT DETECTION


Running CPD: 100%|██████████| 54/54 [18:20<00:00, 20.39s/it]



✓ Created 3070 segments

Segment summary:
  Mean: 8.0s ± 6.0s
  Range: 4.0s - 57.2s

STEP 4: TEMPORAL 5-FOLD SPLIT

✓ Created 5-fold split
✓ Total buffer segments: 112

GLOBAL percentages:
  Fold 1: Test=20.4% ✓
  Fold 2: Test=20.8% ✓
  Fold 3: Test=20.8% ✓
  Fold 4: Test=20.6% ✓
  Fold 5: Test=20.7% ✓

✓ ALL FOLDS WITHIN RANGE!

STEP 5: CREATE WINDOWS

Fold 1/5... Train: 6619, Val: 1634
    Train distribution:
      Activity  1:  623 ( 9.4%)
      Activity  2:  551 ( 8.3%)
      Activity  3:  666 (10.1%)
      Activity  4:  694 (10.5%)
      Activity  5:  706 (10.7%)
      Activity  6:  678 (10.2%)
      Activity  7:  687 (10.4%)
      Activity  8:  539 ( 8.1%)
      Activity  9:  711 (10.7%)
      Activity 10:  764 (11.5%)

Fold 2/5... Train: 6522, Val: 1734
    Train distribution:
      Activity  1:  608 ( 9.3%)
      Activity  2:  555 ( 8.5%)
      Activity  3:  661 (10.1%)
      Activity  4:  669 (10.3%)
      Activity  5:  692 (10.6%)
      Activity  6:  677 (10.4%)
      Activi

## EEGNet for 10 class classification

In [4]:
import numpy as np
import pickle
import os
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, classification_report, confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# =====================================================
# GPU CONFIGURATION
# =====================================================

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print("✓ GPU configured with mixed precision")
    except RuntimeError as e:
        print(f"GPU error: {e}")

# =====================================================
# CONFIGURATION
# =====================================================

# Paths
DATA_DIR = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/windowed_zscore_temporal5fold_50overlap_10class'
OUTPUT_DIR = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_10class_zscore_50overlap'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model parameters (same as before)
CHANNELS = 16
SAMPLES = 500
DROPOUT = 0.5

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
PATIENCE = 15

print("="*70)
print("EEGNET: 10-CLASS CLASSIFICATION")
print("="*70)

print(f"\nConfiguration:")
print(f"  Data: {DATA_DIR}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Classes: 10 (multi-class)")

print(f"\nTraining parameters:")
print(f"  Epochs: {EPOCHS}, LR: {LEARNING_RATE}, Patience: {PATIENCE}")

# =====================================================
# LOAD CV SPLITS
# =====================================================

print(f"\n" + "="*70)
print("LOADING CV SPLITS")
print("="*70)

with open(os.path.join(DATA_DIR, 'cv_splits.pkl'), 'rb') as f:
    cv_splits = pickle.load(f)

with open(os.path.join(DATA_DIR, 'dataset_metadata.pkl'), 'rb') as f:
    metadata = pickle.load(f)

N_CLASSES = metadata['n_classes']
ACTIVITY_IDS = metadata['activity_ids']

print(f"✓ Loaded {len(cv_splits)} folds")
print(f"  Number of classes: {N_CLASSES}")
print(f"  Activity IDs: {ACTIVITY_IDS}")

for fold_data in cv_splits:
    fold_num = fold_data['fold']
    print(f"\n  Fold {fold_num}:")
    print(f"    Train: {len(fold_data['y_train'])} windows")
    print(f"    Val:   {len(fold_data['y_val'])} windows")

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def compute_metrics_multiclass(y_true, y_pred, y_pred_proba):
    """Compute comprehensive metrics for multi-class classification."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'macro_precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'macro_recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'macro_f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
        'weighted_precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'weighted_recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'weighted_f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }
    
    # Per-class metrics
    per_class_f1 = f1_score(y_true, y_pred, average=None, zero_division=0)
    metrics['per_class_f1'] = {int(aid): float(f1) for aid, f1 in zip(ACTIVITY_IDS, per_class_f1)}
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    metrics['confusion_matrix'] = cm
    
    return metrics


def compute_weighted_metrics(fold_results):
    """Compute weighted metrics across folds."""
    total_test = sum(r['n_test_windows'] for r in fold_results)
    weights = np.array([r['n_test_windows'] / total_test for r in fold_results])
    
    metrics = {}
    for metric_name in ['accuracy', 'macro_f1', 'weighted_f1', 'macro_precision', 
                       'macro_recall', 'weighted_precision', 'weighted_recall']:
        values = np.array([r['metrics'][metric_name] for r in fold_results])
        
        weighted_mean = np.sum(values * weights)
        weighted_var = np.sum(weights * (values - weighted_mean)**2)
        weighted_std = np.sqrt(weighted_var)
        
        metrics[metric_name] = {
            'mean': weighted_mean,
            'std': weighted_std
        }
    
    return metrics

# =====================================================
# MODEL BUILDING
# =====================================================

def build_eegnet(n_classes=10, channels=16, samples=500, dropout=0.5):
    """Build EEGNet for multi-class classification."""
    F1, D, F2 = 8, 2, 16
    kernel_length = 64
    
    input_shape = (samples, channels, 1)
    input_layer = layers.Input(shape=input_shape)
    
    # Block 1
    block1 = layers.Conv2D(F1, (kernel_length, 1), padding='same', use_bias=False)(input_layer)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.DepthwiseConv2D((1, channels), use_bias=False, depth_multiplier=D,
                                   depthwise_constraint=keras.constraints.max_norm(1.))(block1)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.Activation('elu')(block1)
    block1 = layers.AveragePooling2D((4, 1))(block1)
    block1 = layers.Dropout(dropout)(block1)
    
    # Block 2
    block2 = layers.SeparableConv2D(F2, (16, 1), use_bias=False, padding='same')(block1)
    block2 = layers.BatchNormalization()(block2)
    block2 = layers.Activation('elu')(block2)
    block2 = layers.AveragePooling2D((8, 1))(block2)
    block2 = layers.Dropout(dropout)(block2)
    
    # Classification
    flatten = layers.Flatten()(block2)
    dense = layers.Dense(n_classes, kernel_constraint=keras.constraints.max_norm(0.25))(flatten)
    softmax = layers.Activation('softmax')(dense)
    
    model = models.Model(inputs=input_layer, outputs=softmax)
    
    return model


def train_model(model, X_train, y_train, X_val, y_val, lr, epochs, fold_name, use_class_weights=True):
    """Train model and return results."""
    
    X_train_reshaped = X_train.reshape(len(X_train), SAMPLES, CHANNELS, 1)
    X_val_reshaped = X_val.reshape(len(X_val), SAMPLES, CHANNELS, 1)
    
    y_train_cat = to_categorical(y_train - 1, N_CLASSES)  # Convert 1-10 to 0-9
    y_val_cat = to_categorical(y_val - 1, N_CLASSES)
    
    class_weight_dict = None
    if use_class_weights:
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
        print(f"    Class weights computed for {len(class_weight_dict)} classes")
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=0
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=0
    )
    
    history = model.fit(
        X_train_reshaped, y_train_cat,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val_cat),
        class_weight=class_weight_dict,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    y_pred_proba = model.predict(X_val_reshaped, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1) + 1  # Convert back to 1-10
    
    metrics = compute_metrics_multiclass(y_val, y_pred, y_pred_proba)
    
    print(f"    Accuracy:     {metrics['accuracy']:.4f}")
    print(f"    Macro F1:     {metrics['macro_f1']:.4f}")
    print(f"    Weighted F1:  {metrics['weighted_f1']:.4f}")
    
    return {
        'history': history.history,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'metrics': metrics,
        'epochs_trained': len(history.history['loss']),
        'n_test_windows': len(y_val),
        'class_weights': class_weight_dict
    }

# =====================================================
# EXPERIMENT: EEGNET 10-CLASS
# =====================================================

print(f"\n" + "="*70)
print("EXPERIMENT: EEGNET 10-CLASS")
print("="*70)

exp_results = []

for fold_data in cv_splits:
    fold_num = fold_data['fold']
    
    print(f"\nFold {fold_num}/{len(cv_splits)}...")
    
    X_train = fold_data['X_train']
    y_train = fold_data['y_train']
    X_val = fold_data['X_val']
    y_val = fold_data['y_val']
    
    print(f"  Building model...")
    model = build_eegnet(n_classes=N_CLASSES, channels=CHANNELS, 
                        samples=SAMPLES, dropout=DROPOUT)
    
    print(f"  Training...")
    results = train_model(model, X_train, y_train, X_val, y_val,
                         lr=LEARNING_RATE, epochs=EPOCHS, 
                         fold_name=f'eegnet_10class_fold{fold_num}',
                         use_class_weights=True)
    
    results['fold'] = fold_num
    exp_results.append(results)
    
    fold_dir = os.path.join(OUTPUT_DIR, f'fold_{fold_num}')
    os.makedirs(fold_dir, exist_ok=True)
    model.save(os.path.join(fold_dir, 'eegnet_10class.h5'))
    print(f"  ✓ Saved model")
    
    del model
    tf.keras.backend.clear_session()
    gc.collect()

# Compute averages
exp_avg_standard = {
    'accuracy': np.mean([r['metrics']['accuracy'] for r in exp_results]),
    'macro_f1': np.mean([r['metrics']['macro_f1'] for r in exp_results]),
    'weighted_f1': np.mean([r['metrics']['weighted_f1'] for r in exp_results]),
    'macro_precision': np.mean([r['metrics']['macro_precision'] for r in exp_results]),
    'macro_recall': np.mean([r['metrics']['macro_recall'] for r in exp_results]),
    'weighted_precision': np.mean([r['metrics']['weighted_precision'] for r in exp_results]),
    'weighted_recall': np.mean([r['metrics']['weighted_recall'] for r in exp_results]),
    'std_accuracy': np.std([r['metrics']['accuracy'] for r in exp_results]),
    'std_macro_f1': np.std([r['metrics']['macro_f1'] for r in exp_results]),
    'std_weighted_f1': np.std([r['metrics']['weighted_f1'] for r in exp_results]),
}

exp_avg_weighted = compute_weighted_metrics(exp_results)

print(f"\n✓ Experiment Complete:")
print(f"\n  Standard Mean:")
print(f"    Accuracy:        {exp_avg_standard['accuracy']:.4f} ± {exp_avg_standard['std_accuracy']:.4f}")
print(f"    Macro F1:        {exp_avg_standard['macro_f1']:.4f} ± {exp_avg_standard['std_macro_f1']:.4f}")
print(f"    Weighted F1:     {exp_avg_standard['weighted_f1']:.4f} ± {exp_avg_standard['std_weighted_f1']:.4f}")
print(f"    Macro Precision: {exp_avg_standard['macro_precision']:.4f}")
print(f"    Macro Recall:    {exp_avg_standard['macro_recall']:.4f}")

print(f"\n  Weighted Mean:")
print(f"    Accuracy:        {exp_avg_weighted['accuracy']['mean']:.4f} ± {exp_avg_weighted['accuracy']['std']:.4f}")
print(f"    Macro F1:        {exp_avg_weighted['macro_f1']['mean']:.4f} ± {exp_avg_weighted['macro_f1']['std']:.4f}")
print(f"    Weighted F1:     {exp_avg_weighted['weighted_f1']['mean']:.4f} ± {exp_avg_weighted['weighted_f1']['std']:.4f}")

# =====================================================
# AGGREGATE CONFUSION MATRIX
# =====================================================

print(f"\n" + "="*70)
print("AGGREGATE CONFUSION MATRIX")
print("="*70)

# Sum confusion matrices across folds
cm_sum = np.zeros((N_CLASSES, N_CLASSES))
for result in exp_results:
    cm_sum += result['metrics']['confusion_matrix']

# Normalize
cm_normalized = cm_sum / cm_sum.sum(axis=1, keepdims=True)

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=ACTIVITY_IDS, yticklabels=ACTIVITY_IDS)
plt.xlabel('Predicted Activity', fontsize=12)
plt.ylabel('True Activity', fontsize=12)
plt.title('Normalized Confusion Matrix (Aggregated Across Folds)', fontsize=14, fontweight='bold')
plt.tight_layout()
cm_file = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
plt.savefig(cm_file, dpi=150, bbox_inches='tight')
print(f"✓ Saved confusion matrix: {cm_file}")
plt.close()

# =====================================================
# PER-CLASS F1 SCORES
# =====================================================

print(f"\n" + "="*70)
print("PER-CLASS F1 SCORES")
print("="*70)

# Average per-class F1 across folds
per_class_f1_avg = {}
for aid in ACTIVITY_IDS:
    f1_scores = [r['metrics']['per_class_f1'][aid] for r in exp_results]
    per_class_f1_avg[aid] = {
        'mean': np.mean(f1_scores),
        'std': np.std(f1_scores)
    }

print(f"\nActivity-wise F1 scores:")
for aid in ACTIVITY_IDS:
    mean_f1 = per_class_f1_avg[aid]['mean']
    std_f1 = per_class_f1_avg[aid]['std']
    print(f"  Activity {aid:2d}: {mean_f1:.4f} ± {std_f1:.4f}")

# =====================================================
# SAVE RESULTS
# =====================================================

print(f"\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

all_results = {
    'model': 'EEGNet',
    'n_classes': N_CLASSES,
    'activity_ids': ACTIVITY_IDS,
    'overlap_version': '50%',
    'preprocessing': 'z_score_normalization',
    'cv_strategy': 'temporal_5fold',
    'exp_results': exp_results,
    'exp_avg_standard': exp_avg_standard,
    'exp_avg_weighted': exp_avg_weighted,
    'per_class_f1_avg': per_class_f1_avg,
    'confusion_matrix_aggregated': cm_sum.tolist(),
    'confusion_matrix_normalized': cm_normalized.tolist(),
    'timestamp': timestamp,
    'config': {
        'data_dir': DATA_DIR,
        'n_folds': len(cv_splits),
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'patience': PATIENCE,
        'dropout': DROPOUT
    }
}

results_file = os.path.join(OUTPUT_DIR, f'eegnet_10class_results_{timestamp}.pkl')
with open(results_file, 'wb') as f:
    pickle.dump(all_results, f)

print(f"✓ Saved: {results_file}")

print(f"\n" + "="*70)
print("COMPLETED SUCCESSFULLY")
print("="*70)
print(f"\nResults saved to: {OUTPUT_DIR}")

2025-12-16 13:55:29.479643: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


✓ GPU configured with mixed precision
EEGNET: 10-CLASS CLASSIFICATION

Configuration:
  Data: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/windowed_zscore_temporal5fold_50overlap_10class
  Output: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_10class_zscore_50overlap
  Classes: 10 (multi-class)

Training parameters:
  Epochs: 100, LR: 0.001, Patience: 15

LOADING CV SPLITS
✓ Loaded 5 folds
  Number of classes: 10
  Activity IDs: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]

  Fold 1:
    Train: 6619 windows
    Val:   1634 windows

  Fold 2:
    Train: 6522 windows
    Val:   1734 windows

  Fold 3:
    Train: 6484 windows
    Val:   1751 windows

  Fold 4:
    Train: 6476 windows
    Val:   1739 windows

  Fold 5:
    Train: 6426 windows
    Val:   1745 windows

EXPERIMENT: EEGNET 10-CLASS

Fold 1/5...
  Building model...


I0000 00:00:1765911331.756382  570792 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14601 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:0c:00.0, compute capability: 8.6
I0000 00:00:1765911331.756788  570792 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 23783 MB memory:  -> device: 1, name: Quadro P6000, pci bus id: 0000:0b:00.0, compute capability: 6.1


  Training...
    Class weights computed for 10 classes


2025-12-16 13:55:33.650526: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f6104805420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-16 13:55:33.650541: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-12-16 13:55:33.650545: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2025-12-16 13:55:33.676685: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-16 13:55:33.878001: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
I0000 00:00:1765911336.732865  571376 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    Accuracy:     0.5208
    Macro F1:     0.5302
    Weighted F1:  0.5240
  ✓ Saved model

Fold 2/5...
  Building model...
  Training...
    Class weights computed for 10 classes


    Accuracy:     0.7751
    Macro F1:     0.7755
    Weighted F1:  0.7744
  ✓ Saved model

Fold 3/5...
  Building model...
  Training...
    Class weights computed for 10 classes


    Accuracy:     0.8058
    Macro F1:     0.8087
    Weighted F1:  0.8054
  ✓ Saved model

Fold 4/5...
  Building model...
  Training...
    Class weights computed for 10 classes


    Accuracy:     0.8194
    Macro F1:     0.8228
    Weighted F1:  0.8206
  ✓ Saved model

Fold 5/5...
  Building model...
  Training...
    Class weights computed for 10 classes


    Accuracy:     0.7874
    Macro F1:     0.7916
    Weighted F1:  0.7869
  ✓ Saved model

✓ Experiment Complete:

  Standard Mean:
    Accuracy:        0.7417 ± 0.1115
    Macro F1:        0.7458 ± 0.1089
    Weighted F1:     0.7423 ± 0.1103
    Macro Precision: 0.7565
    Macro Recall:    0.7475

  Weighted Mean:
    Accuracy:        0.7445 ± 0.1094
    Macro F1:        0.7485 ± 0.1069
    Weighted F1:     0.7450 ± 0.1082

AGGREGATE CONFUSION MATRIX
✓ Saved confusion matrix: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_10class_zscore_50overlap/confusion_matrix.png

PER-CLASS F1 SCORES

Activity-wise F1 scores:
  Activity  1: 0.7259 ± 0.1024
  Activity  2: 0.7147 ± 0.1140
  Activity  3: 0.7791 ± 0.1270
  Activity  4: 0.8643 ± 0.0494
  Activity  5: 0.7389 ± 0.1420
  Activity  6: 0.7438 ± 0.1116
  Activity  7: 0.6555 ± 0.1293
  Activity  8: 0.8672 ± 0.1219
  Activity  9: 0.6793 ± 0.1233
  Activity 10: 0.6889 ± 0.1139

SAVING RESULTS
✓ Saved: /home/jupyter-yin10/EEG_H

## EEGNet + LSTM for 10 class classificaiton

In [5]:
import numpy as np
import pickle
import os
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    f1_score, classification_report, confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# =====================================================
# GPU CONFIGURATION
# =====================================================

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Disable mixed precision for LSTM (avoids slowdown)
        tf.keras.mixed_precision.set_global_policy('float32')
        print("✓ GPU configured (float32 for LSTM speed)")
    except RuntimeError as e:
        print(f"GPU error: {e}")

# =====================================================
# CONFIGURATION
# =====================================================

# Paths
DATA_DIR = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/windowed_zscore_temporal5fold_50overlap_10class'
OUTPUT_DIR = '/home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_lstm_10class_zscore_50overlap'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model parameters (same as before)
CHANNELS = 16
SAMPLES = 500
DROPOUT = 0.5

# LSTM parameters (same as before)
LSTM_UNITS_1 = 128
LSTM_UNITS_2 = 64
LSTM_DROPOUT = 0.3

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
PATIENCE = 15

print("="*70)
print("EEGNET-LSTM: 10-CLASS CLASSIFICATION")
print("="*70)

print(f"\nConfiguration:")
print(f"  Data: {DATA_DIR}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Classes: 10 (multi-class)")

print(f"\nModel parameters:")
print(f"  EEGNet: F1=8, D=2, F2=16, dropout={DROPOUT}")
print(f"  LSTM: [{LSTM_UNITS_1}, {LSTM_UNITS_2}] units, dropout={LSTM_DROPOUT}")

print(f"\nTraining parameters:")
print(f"  Epochs: {EPOCHS}, LR: {LEARNING_RATE}, Patience: {PATIENCE}")

# =====================================================
# LOAD CV SPLITS
# =====================================================

print(f"\n" + "="*70)
print("LOADING CV SPLITS")
print("="*70)

with open(os.path.join(DATA_DIR, 'cv_splits.pkl'), 'rb') as f:
    cv_splits = pickle.load(f)

with open(os.path.join(DATA_DIR, 'dataset_metadata.pkl'), 'rb') as f:
    metadata = pickle.load(f)

N_CLASSES = metadata['n_classes']
ACTIVITY_IDS = metadata['activity_ids']

print(f"✓ Loaded {len(cv_splits)} folds")
print(f"  Number of classes: {N_CLASSES}")
print(f"  Activity IDs: {ACTIVITY_IDS}")

for fold_data in cv_splits:
    fold_num = fold_data['fold']
    print(f"\n  Fold {fold_num}:")
    print(f"    Train: {len(fold_data['y_train'])} windows")
    print(f"    Val:   {len(fold_data['y_val'])} windows")

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def compute_metrics_multiclass(y_true, y_pred, y_pred_proba):
    """Compute comprehensive metrics for multi-class classification."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'macro_precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'macro_recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'macro_f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
        'weighted_precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'weighted_recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'weighted_f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }
    
    # Per-class metrics
    per_class_f1 = f1_score(y_true, y_pred, average=None, zero_division=0)
    metrics['per_class_f1'] = {int(aid): float(f1) for aid, f1 in zip(ACTIVITY_IDS, per_class_f1)}
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    metrics['confusion_matrix'] = cm
    
    return metrics


def compute_weighted_metrics(fold_results):
    """Compute weighted metrics across folds."""
    total_test = sum(r['n_test_windows'] for r in fold_results)
    weights = np.array([r['n_test_windows'] / total_test for r in fold_results])
    
    metrics = {}
    for metric_name in ['accuracy', 'macro_f1', 'weighted_f1', 'macro_precision', 
                       'macro_recall', 'weighted_precision', 'weighted_recall']:
        values = np.array([r['metrics'][metric_name] for r in fold_results])
        
        weighted_mean = np.sum(values * weights)
        weighted_var = np.sum(weights * (values - weighted_mean)**2)
        weighted_std = np.sqrt(weighted_var)
        
        metrics[metric_name] = {
            'mean': weighted_mean,
            'std': weighted_std
        }
    
    return metrics

# =====================================================
# MODEL BUILDING
# =====================================================

def build_eegnet_lstm(n_classes=10, channels=16, samples=500, 
                      eegnet_dropout=0.5, lstm_dropout=0.3,
                      lstm_units_1=128, lstm_units_2=64):
    """
    Build EEGNet-LSTM architecture for multi-class classification.
    
    CRITICAL: Use recurrent_dropout=0.0 for CuDNN fast path!
    Use SpatialDropout1D instead for regularization.
    """
    F1, D, F2 = 8, 2, 16
    kernel_length = 64
    
    input_shape = (samples, channels, 1)
    input_layer = layers.Input(shape=input_shape)
    
    # ==================== EEGNet Block 1 ====================
    block1 = layers.Conv2D(F1, (kernel_length, 1), padding='same', use_bias=False)(input_layer)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.DepthwiseConv2D((1, channels), use_bias=False, depth_multiplier=D,
                                   depthwise_constraint=keras.constraints.max_norm(1.))(block1)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.Activation('elu')(block1)
    block1 = layers.AveragePooling2D((4, 1))(block1)
    block1 = layers.Dropout(eegnet_dropout)(block1)
    
    # ==================== EEGNet Block 2 ====================
    block2 = layers.SeparableConv2D(F2, (16, 1), use_bias=False, padding='same')(block1)
    block2 = layers.BatchNormalization()(block2)
    block2 = layers.Activation('elu')(block2)
    block2 = layers.AveragePooling2D((8, 1))(block2)
    block2 = layers.Dropout(eegnet_dropout)(block2)
    
    # ==================== Reshape for LSTM ====================
    # Output shape: (batch, time_steps, features)
    # block2 shape: (batch, 15, 1, 16) → reshape to (batch, 15, 16)
    reshape = layers.Reshape((15, 16))(block2)
    
    # ==================== LSTM Layers ====================
    # CRITICAL: recurrent_dropout=0.0 for CuDNN speed!
    lstm1 = layers.LSTM(
        lstm_units_1,
        return_sequences=True,
        dropout=0.0,  # Use SpatialDropout1D instead
        recurrent_dropout=0.0,  # MUST be 0 for CuDNN!
        name='lstm_1'
    )(reshape)
    lstm1 = layers.SpatialDropout1D(lstm_dropout)(lstm1)  # Dropout after LSTM
    
    lstm2 = layers.LSTM(
        lstm_units_2,
        return_sequences=False,
        dropout=0.0,  # Use dropout after instead
        recurrent_dropout=0.0,  # MUST be 0 for CuDNN!
        name='lstm_2'
    )(lstm1)
    lstm2 = layers.Dropout(lstm_dropout)(lstm2)  # Dropout after LSTM
    
    # ==================== Classification ====================
    dense = layers.Dense(n_classes, kernel_constraint=keras.constraints.max_norm(0.25))(lstm2)
    softmax = layers.Activation('softmax')(dense)
    
    model = models.Model(inputs=input_layer, outputs=softmax)
    
    return model


def train_model(model, X_train, y_train, X_val, y_val, lr, epochs, fold_name, use_class_weights=True):
    """Train model and return results."""
    
    X_train_reshaped = X_train.reshape(len(X_train), SAMPLES, CHANNELS, 1)
    X_val_reshaped = X_val.reshape(len(X_val), SAMPLES, CHANNELS, 1)
    
    y_train_cat = to_categorical(y_train - 1, N_CLASSES)  # Convert 1-10 to 0-9
    y_val_cat = to_categorical(y_val - 1, N_CLASSES)
    
    class_weight_dict = None
    if use_class_weights:
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
        print(f"    Class weights computed for {len(class_weight_dict)} classes")
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=0
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=0
    )
    
    history = model.fit(
        X_train_reshaped, y_train_cat,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val_cat),
        class_weight=class_weight_dict,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    y_pred_proba = model.predict(X_val_reshaped, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1) + 1  # Convert back to 1-10
    
    metrics = compute_metrics_multiclass(y_val, y_pred, y_pred_proba)
    
    print(f"    Accuracy:     {metrics['accuracy']:.4f}")
    print(f"    Macro F1:     {metrics['macro_f1']:.4f}")
    print(f"    Weighted F1:  {metrics['weighted_f1']:.4f}")
    
    return {
        'history': history.history,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'metrics': metrics,
        'epochs_trained': len(history.history['loss']),
        'n_test_windows': len(y_val),
        'class_weights': class_weight_dict
    }

# =====================================================
# EXPERIMENT: EEGNET-LSTM 10-CLASS
# =====================================================

print(f"\n" + "="*70)
print("EXPERIMENT: EEGNET-LSTM 10-CLASS")
print("="*70)

exp_results = []

for fold_data in cv_splits:
    fold_num = fold_data['fold']
    
    print(f"\nFold {fold_num}/{len(cv_splits)}...")
    
    X_train = fold_data['X_train']
    y_train = fold_data['y_train']
    X_val = fold_data['X_val']
    y_val = fold_data['y_val']
    
    print(f"  Building EEGNet-LSTM model...")
    model = build_eegnet_lstm(
        n_classes=N_CLASSES, 
        channels=CHANNELS, 
        samples=SAMPLES, 
        eegnet_dropout=DROPOUT,
        lstm_dropout=LSTM_DROPOUT,
        lstm_units_1=LSTM_UNITS_1,
        lstm_units_2=LSTM_UNITS_2
    )
    
    print(f"    Total params: {model.count_params():,}")
    
    print(f"  Training...")
    results = train_model(model, X_train, y_train, X_val, y_val,
                         lr=LEARNING_RATE, epochs=EPOCHS, 
                         fold_name=f'eegnet_lstm_10class_fold{fold_num}',
                         use_class_weights=True)
    
    results['fold'] = fold_num
    exp_results.append(results)
    
    fold_dir = os.path.join(OUTPUT_DIR, f'fold_{fold_num}')
    os.makedirs(fold_dir, exist_ok=True)
    model.save(os.path.join(fold_dir, 'eegnet_lstm_10class.h5'))
    print(f"  ✓ Saved model")
    
    del model
    tf.keras.backend.clear_session()
    gc.collect()

# Compute averages
exp_avg_standard = {
    'accuracy': np.mean([r['metrics']['accuracy'] for r in exp_results]),
    'macro_f1': np.mean([r['metrics']['macro_f1'] for r in exp_results]),
    'weighted_f1': np.mean([r['metrics']['weighted_f1'] for r in exp_results]),
    'macro_precision': np.mean([r['metrics']['macro_precision'] for r in exp_results]),
    'macro_recall': np.mean([r['metrics']['macro_recall'] for r in exp_results]),
    'weighted_precision': np.mean([r['metrics']['weighted_precision'] for r in exp_results]),
    'weighted_recall': np.mean([r['metrics']['weighted_recall'] for r in exp_results]),
    'std_accuracy': np.std([r['metrics']['accuracy'] for r in exp_results]),
    'std_macro_f1': np.std([r['metrics']['macro_f1'] for r in exp_results]),
    'std_weighted_f1': np.std([r['metrics']['weighted_f1'] for r in exp_results]),
}

exp_avg_weighted = compute_weighted_metrics(exp_results)

print(f"\n✓ Experiment Complete:")
print(f"\n  Standard Mean:")
print(f"    Accuracy:        {exp_avg_standard['accuracy']:.4f} ± {exp_avg_standard['std_accuracy']:.4f}")
print(f"    Macro F1:        {exp_avg_standard['macro_f1']:.4f} ± {exp_avg_standard['std_macro_f1']:.4f}")
print(f"    Weighted F1:     {exp_avg_standard['weighted_f1']:.4f} ± {exp_avg_standard['std_weighted_f1']:.4f}")
print(f"    Macro Precision: {exp_avg_standard['macro_precision']:.4f}")
print(f"    Macro Recall:    {exp_avg_standard['macro_recall']:.4f}")

print(f"\n  Weighted Mean:")
print(f"    Accuracy:        {exp_avg_weighted['accuracy']['mean']:.4f} ± {exp_avg_weighted['accuracy']['std']:.4f}")
print(f"    Macro F1:        {exp_avg_weighted['macro_f1']['mean']:.4f} ± {exp_avg_weighted['macro_f1']['std']:.4f}")
print(f"    Weighted F1:     {exp_avg_weighted['weighted_f1']['mean']:.4f} ± {exp_avg_weighted['weighted_f1']['std']:.4f}")

# =====================================================
# AGGREGATE CONFUSION MATRIX
# =====================================================

print(f"\n" + "="*70)
print("AGGREGATE CONFUSION MATRIX")
print("="*70)

# Sum confusion matrices across folds
cm_sum = np.zeros((N_CLASSES, N_CLASSES))
for result in exp_results:
    cm_sum += result['metrics']['confusion_matrix']

# Normalize
cm_normalized = cm_sum / cm_sum.sum(axis=1, keepdims=True)

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=ACTIVITY_IDS, yticklabels=ACTIVITY_IDS)
plt.xlabel('Predicted Activity', fontsize=12)
plt.ylabel('True Activity', fontsize=12)
plt.title('Normalized Confusion Matrix (Aggregated Across Folds)', fontsize=14, fontweight='bold')
plt.tight_layout()
cm_file = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
plt.savefig(cm_file, dpi=150, bbox_inches='tight')
print(f"✓ Saved confusion matrix: {cm_file}")
plt.close()

# =====================================================
# PER-CLASS F1 SCORES
# =====================================================

print(f"\n" + "="*70)
print("PER-CLASS F1 SCORES")
print("="*70)

# Average per-class F1 across folds
per_class_f1_avg = {}
for aid in ACTIVITY_IDS:
    f1_scores = [r['metrics']['per_class_f1'][aid] for r in exp_results]
    per_class_f1_avg[aid] = {
        'mean': np.mean(f1_scores),
        'std': np.std(f1_scores)
    }

print(f"\nActivity-wise F1 scores:")
for aid in ACTIVITY_IDS:
    mean_f1 = per_class_f1_avg[aid]['mean']
    std_f1 = per_class_f1_avg[aid]['std']
    print(f"  Activity {aid:2d}: {mean_f1:.4f} ± {std_f1:.4f}")

# =====================================================
# SAVE RESULTS
# =====================================================

print(f"\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

all_results = {
    'model': 'EEGNet-LSTM',
    'n_classes': N_CLASSES,
    'activity_ids': ACTIVITY_IDS,
    'overlap_version': '50%',
    'preprocessing': 'z_score_normalization',
    'cv_strategy': 'temporal_5fold',
    'exp_results': exp_results,
    'exp_avg_standard': exp_avg_standard,
    'exp_avg_weighted': exp_avg_weighted,
    'per_class_f1_avg': per_class_f1_avg,
    'confusion_matrix_aggregated': cm_sum.tolist(),
    'confusion_matrix_normalized': cm_normalized.tolist(),
    'timestamp': timestamp,
    'config': {
        'data_dir': DATA_DIR,
        'n_folds': len(cv_splits),
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'patience': PATIENCE,
        'eegnet_dropout': DROPOUT,
        'lstm_dropout': LSTM_DROPOUT,
        'lstm_units': [LSTM_UNITS_1, LSTM_UNITS_2]
    }
}

results_file = os.path.join(OUTPUT_DIR, f'eegnet_lstm_10class_results_{timestamp}.pkl')
with open(results_file, 'wb') as f:
    pickle.dump(all_results, f)

print(f"✓ Saved: {results_file}")

print(f"\n" + "="*70)
print("COMPLETED SUCCESSFULLY")
print("="*70)
print(f"\nResults saved to: {OUTPUT_DIR}")

✓ GPU configured (float32 for LSTM speed)
EEGNET-LSTM: 10-CLASS CLASSIFICATION

Configuration:
  Data: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/data/windowed_zscore_temporal5fold_50overlap_10class
  Output: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_lstm_10class_zscore_50overlap
  Classes: 10 (multi-class)

Model parameters:
  EEGNet: F1=8, D=2, F2=16, dropout=0.5
  LSTM: [128, 64] units, dropout=0.3

Training parameters:
  Epochs: 100, LR: 0.001, Patience: 15

LOADING CV SPLITS
✓ Loaded 5 folds
  Number of classes: 10
  Activity IDs: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]

  Fold 1:
    Train: 6619 windows
    Val:   1634 windows

  Fold 2:
    Train: 6522 windows
    Val:   1734 windows

  Fold 3:
    Train: 6484 windows
    Val:   1751 windows

  Fold 4:
    Train: 6476 windows
    Val:   1739 windows

  Fold 5:
    Train: 6426 windows
    Val:   1745 windows

EX

E0000 00:00:1765911718.850784  570792 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


    Accuracy:     0.7436
    Macro F1:     0.7375
    Weighted F1:  0.7424
  ✓ Saved model

Fold 2/5...
  Building EEGNet-LSTM model...
    Total params: 125,738
  Training...
    Class weights computed for 10 classes


E0000 00:00:1765911926.838309  570792 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


    Accuracy:     0.9556
    Macro F1:     0.9553
    Weighted F1:  0.9557
  ✓ Saved model

Fold 3/5...
  Building EEGNet-LSTM model...
    Total params: 125,738
  Training...
    Class weights computed for 10 classes


...

    Accuracy:     0.9423
    Macro F1:     0.9405
    Weighted F1:  0.9422
  ✓ Saved model

Fold 4/5...
  Building EEGNet-LSTM model...
    Total params: 125,738
  Training...
    Class weights computed for 10 classes


E0000 00:00:1765912330.381202  570792 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


    Accuracy:     0.9770
    Macro F1:     0.9773
    Weighted F1:  0.9770
  ✓ Saved model

Fold 5/5...
  Building EEGNet-LSTM model...
    Total params: 125,738
  Training...
    Class weights computed for 10 classes


E0000 00:00:1765912743.804209  570792 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


    Accuracy:     0.9817
    Macro F1:     0.9812
    Weighted F1:  0.9817
  ✓ Saved model

✓ Experiment Complete:

  Standard Mean:
    Accuracy:        0.9200 ± 0.0894
    Macro F1:        0.9184 ± 0.0916
    Weighted F1:     0.9198 ± 0.0899
    Macro Precision: 0.9199
    Macro Recall:    0.9181

  Weighted Mean:
    Accuracy:        0.9222 ± 0.0877
    Macro F1:        0.9206 ± 0.0899
    Weighted F1:     0.9220 ± 0.0882

AGGREGATE CONFUSION MATRIX
✓ Saved confusion matrix: /home/jupyter-yin10/EEG_HAR/Pipeline_experiments/results/eegnet_lstm_10class_zscore_50overlap/confusion_matrix.png

PER-CLASS F1 SCORES

Activity-wise F1 scores:
  Activity  1: 0.9189 ± 0.0848
  Activity  2: 0.8904 ± 0.1124
  Activity  3: 0.9183 ± 0.0905
  Activity  4: 0.8999 ± 0.1265
  Activity  5: 0.9520 ± 0.0556
  Activity  6: 0.9220 ± 0.0572
  Activity  7: 0.9422 ± 0.0788
  Activity  8: 0.9019 ± 0.1280
  Activity  9: 0.9111 ± 0.1042
  Activity 10: 0.9269 ± 0.0958

SAVING RESULTS
✓ Saved: /home/jupyter-yin10/